# Installing Libraries

In [ ]:
#!pip install pandas_datareader

In [ ]:
#!pip install yfinance

In [ ]:
#!pip install yfinance --upgrade --no-cache-dir


In [ ]:
#!pip install html5lib

# Importing Libraries

In [132]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import re
import os
import yfinance as yf
from time import sleep
import glob
import regex as re
import datetime
import sqlalchemy as db
from dotenv import load_dotenv

# Gathering Data - Brazilian Companies

### List of brazilian companies and composition of indexes - Ibovespa, IBrX100, IBrX50, IBrA

## Selenium

Used to get list of companies in brazilian's index

### Configurating WebDriver

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

### Configurating download file

In [3]:
chromeOptions = webdriver.ChromeOptions()
download_path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\stocks_project\data'
prefs = {"download.default_directory" : download_path}
chromeOptions.add_experimental_option("prefs",prefs)

In [ ]:
#Runs the webdriver
driver = webdriver.Chrome(options=chromeOptions)

### Download Index File (.csv)

In [ ]:
def get_index_stocks(index, wait=6):
    '''
        Receives the Index name, download a file that contains the index tickers, and return the name of the downloaded file
    '''

    # Chrome WebDriver opens the index webside
    url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{index.upper()}?language=pt-br'
    driver.get(url)
    driver.implicitly_wait(wait)

    driver.find_element(By.ID, 'segment').send_keys("Setor de Atuação")
    driver.implicitly_wait(wait)
    driver.find_element(By.LINK_TEXT, 'Download').click()
    driver.implicitly_wait(wait)

    # Set the directory
    os.chdir(r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\stocks_project\data')
    sleep(wait)

    # Get the .csv files from the selectec directory and sort them ascending by modification date 
    files = list(glob.glob('*csv'))
    files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

    # Returns the name of the most recent file
    return files[0]

### Create the Index DataFrame

In [ ]:
def create_df(file):
    '''
        This function receives the name of the Stock Exchange index and returns a DataFrame with all companies and their respective sectors
    '''

    # Creating DataFrame
    DataFrame = pd.read_csv(file, 
                                encoding='ISO-8859-1',
                                header=1,                   # Uses line 1 as header
                                sep=';',                    
                                decimal=',',
                                thousands='.',
                                skipfooter=2,               # Removes last 2 lines
                                engine='python',
                                index_col=False)            # Does not make first column as index
    
    # Normalizing columns
    DataFrame.columns = [re.sub('[\.()]', '', re.sub(' ', '_', unidecode(columns.lower()))) for columns in DataFrame.columns]

    return DataFrame

## Creating Index's DataFrames

In [4]:
# Defining file's path
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\stocks_project\data/'

In [ ]:
# Defining DataFrames' columns names
col_names = [
    'sector',
    'ticker',
    'name',
    'type',
    'amount',
    'percentage',
    'percentage_acum'
]

### Ibovespa

In [ ]:
ibov = create_df(get_index_stocks('ibov'))
len(ibov)

In [ ]:
ibov.columns = col_names

In [ ]:
# Save index file
ibov.to_csv(f'{path}IBOV.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

### IBrX100

In [ ]:
ibrx = create_df(get_index_stocks('ibxx'))
len(ibrx)

In [ ]:
ibrx.columns = col_names

In [ ]:
# Save index file
ibrx.to_csv(f'{path}IBRX100.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

### IBrX50

In [ ]:
ibrx50 = create_df(get_index_stocks('ibxl'))
len(ibrx50)

In [ ]:
ibrx50.columns = col_names

In [ ]:
# Save index file
ibrx50.to_csv(f'{path}IBRX50.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

### IBrA

In [ ]:
ibra = create_df(get_index_stocks('ibra'))
len(ibra)

In [ ]:
ibra.columns = col_names

In [ ]:
# Save index file
ibra.to_csv(f'{path}IBRA.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

### Transform Sector - Obsoleto?

In [ ]:
ibra = pd.read_csv(f'{path}IBRA.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.')

In [ ]:
len(ibra)

In [ ]:
ibra

In [ ]:
ibra[['sector_aux', 'sub_sector_aux']] = ibra['sector'].str.split('/',expand=True)

In [ ]:
ibra['sector_aux'] = ibra['sector_aux'].str.strip()
ibra['sub_sector_aux'] = ibra['sub_sector_aux'].str.strip()

In [ ]:
len(ibra['sector_aux'].unique())

In [ ]:
len(ibra['sub_sector_aux'].unique())

## Verificando empresas que divergem nos dois índices - Obsoleto

In [ ]:
emp_ibrx = ibrx['codigo']
set_ibrx = set(emp_ibrx)
len(set_ibrx)

In [ ]:
emp_ibov = ibov['codigo']
set_ibov = set(emp_ibov)
len(set_ibov)

In [ ]:
emp_ibra = ibra['codigo']
set_ibra = set(emp_ibra)
len(set_ibra)

In [ ]:
print(len(set_ibra.difference(set_ibrx)))
print(set_ibra.difference(set_ibrx))

In [ ]:
print(len(set_ibov.difference(set_ibrx)))
print(set_ibov.difference(set_ibrx))

In [ ]:
print(len(set_ibrx.difference(set_ibov)))
print(set_ibrx.difference(set_ibov))

### Verificando qual empresa da lista completa não exista em 2021

In [ ]:
set_codigo = set(lista_codigo)
set_empresas_2021 = set(empresas_2021)

print(len(set_codigo.difference(set_empresas_2021)))
print(set_codigo.difference(set_empresas_2021))

## Criando DataFrame unindo os dois índices - Obsoleto

In [ ]:
df_empresas = pd.concat([ibrx, ibov]).drop_duplicates(subset='codigo', ignore_index=True).drop(['qtde_teorica',r'part_%',r'part_%acum'], axis=1)

# Gathering Data - Brazilian Companies Historical Dataset

### Create data set concatenating historical datasets downloaded from B3 website with data from the companies listed in IBRA Index

## Criando lista com o código dos pricipais ativos

In [ ]:
## Utilizar lista do IBRA
codigo = list(set_ibra)

## Unzip files

In [ ]:
import zipfile
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data/'

i = 1986

while i < 2000:
    with zipfile.ZipFile(f'{path}COTAHIST_A{i}.zip', 'r') as zip_ref:
        zip_ref.extractall(path)
    i +=1



## Rename files

In [ ]:
os.listdir()

In [ ]:
i = 2000

while i < 2022:
    os.rename(f'COTAHIST_A{i}.TXT', f'COTAHIST_A{i}.txt')
    i +=1

## Setting configurations to read B3 historical files

In [ ]:
pd.set_option('display.max_columns', 500)
pd.options.display.max_columns=500

In [ ]:
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\05. Dados\B3\txt/'

year = 2022

widths = [2,8,2,12,3,12,10,3,4,13,13,13,13,13,13,13,5,18,18,13,1,8,7,13,12,3]

col_names = [
"tipo_registro",
"data_pregao",
"cod_bdi",
"cod_negociacao",
"tipo_mercado",
"nome_empresa",
"especificacao_papel",
"prazo_dias_merc_termo",
"moeda_referencia",
"preco_abertura",
"preco_maximo",
"preco_minimo",
"preco_medio",
"preco_ultimo_negocio",
"preco_melhor_oferta_compra",
"preco_melhor_oferta_venda",
"numero_negocios",
"quantidade_papeis_negociados",
"volume_total_negociado",
"preco_exercicio",
"ìndicador_correcao_precos",
"data_vencimento" ,
"fator_cotacao",
"preco_exercicio_pontos",
"codigo_isin",
"num_distribuicao_papel"]

decimal_config=[
"preco_abertura",
"preco_maximo",
"preco_minimo",
"preco_medio",
"preco_ultimo_negocio",
"preco_melhor_oferta_compra",
"preco_melhor_oferta_venda",
"volume_total_negociado",
"preco_exercicio",
"preco_exercicio_pontos"
]


remains = [
"data_pregao",
"cod_negociacao",
"tipo_mercado",
"nome_empresa",
"preco_abertura",
"preco_maximo",
"preco_minimo",
"preco_medio",
"preco_ultimo_negocio",
]

In [ ]:
dataset = pd.DataFrame()

In [ ]:
'''     Concatenate DataFrames
year = 2022
while year > 1985:
    df = pd.read_fwf(f'{path}COTAHIST_A{year}.TXT',
                                    encoding='ISO-8859-1',
                                    header=0,
                                    widths=widths,
                                    skipfooter=1,
                                    engine='python',
                                    parse_dates=[1],
                                    infer_datetime_format=True,
                                    index_col=False)
    year -= 1

    #Definindo nomes das colunas
    df.columns = col_names

    #Corrigindo casas decimais
    for col in decimal_config:
        df[col]=df[col]/100

    #Selecionando colunas
    df = df[remains]
    
    #Mascara de empresas desejadas
    mask_empresas = df.cod_negociacao.isin(codigo)

    df = df[mask_empresas]

    dataset = pd.concat([dataset, df], ignore_index=True)
'''

## Export DF - Historical Series

In [ ]:
dataset.to_csv(f'{path}dataset_IBRA.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
pd.read_csv(f'{path}dataset_IBRA.csv',
                encoding='ISO-8859-1',
                sep=';',
                decimal='.',
                index_col=False
                )

# Gathering Data From Yahoo Finance

### Get historical series of brazilian and american indexes
### Get historical series of Gold, Bitcoin and Ethererum
### Get historical series of american companies

## Yahoo Finance

In [ ]:
# Set YFinance setting

symbol_list_br = ['^BVSP','^IBX50']
symbol_list_eua = ['^DJI','^IXIC','^GSPC','GC=F']
symbol_list_crypto = ['BTC-USD', 'ETH-USD']

name_dict={
    '^BVSP':'Ibovespa',
    '^IBX50':'IBrX50',
    '^DJI':'Dow Jones',
    '^IXIC':'NASDAQ',
    '^GSPC':'S&P 500',
    'GC=F':'Ouro ($)',
    'BTC-USD':'Bitcoin ($)',
    'ETH-USD':'Ethereum ($)'
}

col_names = ['date',
"open",
"high",
"low",
"close",
'cod_yfinance']

drop_col = ['Volume','Dividends','Stock Splits']

start = '2000-01-01'
start_br ='2004-11-03'

In [ ]:
# Create empty indexes DataFrames
'''
df_br = pd.DataFrame()
df_eua = pd.DataFrame()
df_crypto = pd.DataFrame()
'''

In [ ]:
# Fill brazilian indexes historical series dataset
'''
for ativo in symbol_list_br:
        chamada_api = yf.Ticker(ativo).history(period='max')
        chamada_api['cod_yfinace'] = ativo
        df_br = pd.concat([df_br, chamada_api])
'''

In [ ]:
# Tranform data from index to column 0
'''
df_br = df_br.drop(drop_col, axis = 1)
df_br.reset_index(inplace=True)
df_br['Date'] = df_br['Date'].dt.date
df_br.columns = col_names
df_br
'''

In [ ]:
# Put indexes names in the DF
df_br['name'] = df_br['cod_yfinance'].map(name_dict)

In [ ]:
# Reroder columns
df_br = df_br[['date','name','open','high','low','close','cod_yfinance']]

In [ ]:
# Save DataFrame
df_br.to_csv(f'{path}index_br.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
# Read DataFrame
index_br = pd.read_csv(f'{path}index_br.csv',
                    sep = ';',
                    decimal = '.',
                    encoding='ISO-8859-1',
                    index_col=False
)

In [ ]:
index_br

In [ ]:
# Fill american indexes and gold historical series dataset
'''
for ativo in symbol_list_eua:
        chamada_api = yf.Ticker(ativo).history(period='max')
        chamada_api['cod_yfinance'] = ativo
        df_eua = pd.concat([df_eua, chamada_api])
'''

In [ ]:
# Tranform data from index to column 0
'''
df_eua = df_eua.drop(drop_col, axis = 1)
df_eua.reset_index(inplace=True)
df_eua['Date'] = df_eua['Date'].dt.date
df_eua.columns = col_names
df_eua
'''

In [ ]:
# Put indexes names in the DF
df_eua['name'] = df_eua['cod_yfinance'].map(name_dict)

In [ ]:
# Reorder columns
df_eua = df_eua[['date','name','open','high','low','close','cod_yfinance']]

In [ ]:
# Save DataFrame
df_eua.to_csv(f'{path}index_eua.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
# Read DataFrame
index_eua = pd.read_csv(f'{path}index_eua.csv',
                    sep = ';',
                    decimal = '.',
                    encoding='ISO-8859-1',
                    index_col=False)

In [ ]:
index_eua

In [ ]:
# Fill cripto historical series dataset
'''
for ativo in symbol_list_crypto:
        chamada_api = yf.Ticker(ativo).history(period='max')
        chamada_api['cod_yfinace'] = ativo
        df_crypto = pd.concat([df_crypto, chamada_api])
'''

In [ ]:
# Tranform data from index to column 0
'''
df_crypto = df_crypto.drop(drop_col, axis = 1)
df_crypto.reset_index(inplace=True)
df_crypto['Date'] = df_crypto['Date'].dt.date
df_crypto.columns = col_names
df_crypto
'''

In [ ]:
# Put indexes names in the DF
df_crypto['name'] = df_crypto['cod_yfinance'].map(name_dict)

In [ ]:
# Reorder columns
df_crypto = df_crypto[['date','name','open','high','low','close','cod_yfinance']]

In [ ]:
df_crypto.to_csv(f'{path}crypto.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
crypto = pd.read_csv(f'{path}crypto.csv',
                    sep = ';',
                    decimal = '.',
                    encoding='ISO-8859-1',
                    index_col=False)


In [ ]:
crypto

# Gathering Data - American Companies

### List of american companies and composition of indexes - S&P500, Dow Jones, Nasdaq

## S&P500

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

data = pd.read_html(url)

In [ ]:
data[0].head()

In [ ]:
sp500 = data[0].iloc[:,[0,1,3,4]]

In [ ]:
sp500.columns = ['ticker', 'name', 'sector', 'sub_industry']

In [ ]:
sp500

In [ ]:
sp500.to_csv(f'{path}SP500.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
pd.read_csv(f'{path}SP500.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.')

## Nasdaq

In [ ]:
url = 'https://en.wikipedia.org/wiki/Nasdaq-100'

data = pd.read_html(url)

In [ ]:
data[4]
nasdaq = data[4]
nasdaq

In [ ]:
nasdaq.columns = ['name','ticker', 'sector', 'sub_industry']

In [ ]:
nasdaq.to_csv(f'{path}NASDAQ.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
nasdaq = pd.read_csv(f'{path}NASDAQ.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.')

## Dow Jones

In [ ]:
url = 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average'

data = pd.read_html(url)

In [ ]:
data[1]
dow_jones = data[1].iloc[:,[0,2,3]]
dow_jones

In [ ]:
dow_jones.columns = ['name','ticker', 'sector']

In [ ]:
dow_jones.to_csv(f'{path}DOW_JONES.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
dow_jones = pd.read_csv(f'{path}DOW_JONES.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.')

## Create DataFrame with all companies in american indexes

In [ ]:
emp_sp500 = sp500['ticker']
set_sp500 = set(emp_sp500)
len(set_sp500)

In [ ]:
emp_nasdaq = nasdaq['ticker']
set_nasdaq = set(emp_nasdaq)
len(set_nasdaq)

In [ ]:
# Companies in Nasdaq that aren't in sp500
print(len(set_nasdaq.difference(set_sp500)))
print(set_nasdaq.difference(set_sp500))
list_nasdaq = list(set_nasdaq.difference(set_sp500))
list_nasdaq

In [ ]:
# Create DF with all companies in both indexes
sp500_concat = pd.concat([sp500,nasdaq.loc[nasdaq['ticker'].isin(list_nasdaq)]], axis=0, ignore_index=True)

In [ ]:
sp500_concat

In [ ]:
# Save DF
sp500_concat.to_csv(f'{path}eua_all_companies.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

# Gathering Data - Company's Sector and Industry

### Get sector and sub-sector of all companies using WebScrapping on Yahoo Finance website

In [ ]:
ibra = pd.read_csv(f'{path}IBRA.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.')

ibra = ibra[['ticker','name','sector']]
ibra[['sector','sub_industry']] = np.nan

In [ ]:
ibra

In [ ]:
eua = pd.read_csv(f'{path}eua_all_companies.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.')

In [ ]:
eua

In [ ]:
eua.isnull().sum()

In [ ]:
eua[eua['ticker'].str.contains(r'[^a-zA-Z0-9]')]

In [ ]:
# Replace '.' in S&P500 ticker to '-', in order to get the right ticker for yfinance webscrapping
eua['ticker'].replace(r'[^a-zA-Z0-9]', r'-', regex=True, inplace=True)

In [ ]:
#Runs the webdriver
driver = webdriver.Chrome(options=chromeOptions)

In [ ]:
index = 0
wait = 8

for i in eua['ticker']:
    # Get the Yfinance company's url
    url = f'https://finance.yahoo.com/quote/{i}/profile?p={i}'
    driver.get(url)
    driver.implicitly_wait(wait)

    # Get company's sector
    try:
        sector = driver.find_element(By.XPATH, '//*[@id="Col1-0-Profile-Proxy"]/section/div[1]/div/div/p[2]/span[2]')
        driver.implicitly_wait(wait)
        eua.loc[index, 'sector'] = sector.text
    except:
        pass

    # Get company's industry
    try:
        industry = driver.find_element(By.XPATH, '//*[@id="Col1-0-Profile-Proxy"]/section/div[1]/div/div/p[2]/span[4]')
        driver.implicitly_wait(wait)
        eua.loc[index, 'sub_industry'] = industry.text
    except:
        pass

    # Update index
    index +=1    

### Checking if all companies in IBRA index have sector and industry

In [ ]:
ibra[ibra['sector'].isna()]

In [ ]:
ibra['sector'].unique()

In [ ]:
ibra[ibra['sector'] == '']

In [ ]:
ibra.loc[[149,151,200]]

In [ ]:
ibra.loc[149,('sector','sub_industry')] = ['Healthcare', 'Drug Manufacturers—Specialty & Generic']
ibra.loc[151,('sector','sub_industry')] = ['Healthcare', 'Drug Manufacturers—Specialty & Generic']
ibra.loc[200,('sector','sub_industry')] = ['Utilities', 'Utilities—Independent Power Producers']

In [ ]:
ibra.to_csv(f'{path}IBRA_sector.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
ibra = ibra.rename({'sub_industry':'industry'}, axis=1)

In [ ]:
ibra

In [ ]:
cod_yfinance = [f'{i}.SA' for i in ibra['ticker']]

In [ ]:
ibra['cod_yfinance'] = cod_yfinance

In [ ]:
ibra['country'] = 'Brazil'

In [ ]:
ibra

### Checking if all companies in eua DataFrame have sector and industry

In [ ]:
eua.isnull().sum()

In [ ]:
eua['sector'].unique()

In [ ]:
# Save Dataframe
eua.to_csv(f'{path}eua_all_companies.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
eua = eua.rename({'sub_industry':'industry'}, axis = 1)

In [ ]:
eua['cod_yfinance'] = eua['ticker']

In [ ]:
eua['country'] = 'USA'

In [ ]:
eua

# Updating IBRA dataset - Obsoleto - criar novo dataframe com series históricas retiradas do yfinace para todas as empresas

### (dataset that contains brazilian companies's historical data)

# Create Datasets

### Dataset that contains brazilian and american companies's characteristics
### Dataset that contains brazilian and american companies's historical data
### Dataset that contains brazilian and american indexes, gold, dolar (R$), bitcoin and ethereum  characteristics
### Dataset that contains brazilian and american indexes, gold, dolar (R$), bitcoin and ethereum  historical data

In [5]:
# Historical dataset is to big to upload to GitHub
stock_path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\05. Dados\stock_project_datasets/'

## Dataset with all companies's characteristics

In [ ]:
ibra.head()

In [ ]:
eua.head()

In [ ]:
all_companies = pd.concat([ibra, eua], axis=0, ignore_index=True)

In [ ]:
all_companies.to_csv(f'{stock_path}all_companies.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
all_companies = pd.read_csv(f'{stock_path}all_companies_rev4.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.')

In [ ]:
all_companies

## Dataset with indexes, gold, dolar (R$), bitcoin and ethereum  characteristics

In [ ]:
index_ticker = [
    'IBOV'
    ,'IBRX50'
    ,'NASDAQ'
    ,'DOW JONES'
    ,'SP500'
    ,'GOLD'
    ,'BTC'
    ,'ETH'
    ,'USD-BRL'
]

index_name = [
    'Ibovespa'
    ,'IBrX50'
    ,'Nasdaq 100'
    ,'Dow Jones Industrial Average'
    ,'S&P 500'
    ,'Gold (US$)'
    ,'Bitcoin (US$)'
    ,'Ethereum (US$)'
    ,'Dolar (R$)'
]

index_sector = [
    'Index'
    ,'Index'
    ,'Index'
    ,'Index'
    ,'Index'
    ,'Gold'
    ,'Crypto'
    ,'Crypto'
    ,'Currency'
]

index_industry = [
    'Index'
    ,'Index'
    ,'Index'
    ,'Index'
    ,'Index'
    ,'Gold'
    ,'Crypto'
    ,'Crypto'
    ,'Currency'
]

index_cod = [
    '^BVSP'
    ,'^IBX50'
    ,'^DJI'
    ,'NQ=F'
    ,'^GSPC'
    ,'GC=F'
    ,'BTC-USD'
    ,'ETH-USD'
    ,'BRL=X'
]

index_country = [
    'Brazil'
    ,'Brazil'
    ,'USA'
    ,'USA'
    ,'USA'
    ,'USA'
    ,'USA'
    ,'USA'
    ,'Brazil'
]

index_type = [
    'Index'
    ,'Index'
    ,'Index'
    ,'Index'
    ,'Index'
    ,'Gold'
    ,'Crypto'
    ,'Crypto'
    ,'Currency'
]

In [ ]:
df_index = pd.DataFrame()
df_index['ticker'] = index_ticker
df_index['name'] = index_name
df_index['sector'] = index_sector
df_index['industry'] = index_industry
df_index['cod_yfinance'] = index_cod
df_index['country'] = index_country
df_index['type'] = index_type

In [ ]:
df_index

In [ ]:
df_index.to_csv(f'{stock_path}all_indexes.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

## Unificating characteristic datasets

In [ ]:
df_index

In [ ]:
all_companies

In [ ]:
all_companies_and_index = pd.concat([all_companies, df_index], axis=0, ignore_index=True)

In [ ]:
all_companies_and_index

In [ ]:
all_companies_and_index.to_csv(f'{stock_path}all_companies_and_indexes.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

## Dataset with all companies and indexes historical data

In [ ]:
# Create empty DataFrame
historical_data = pd.DataFrame()

In [ ]:
# Creating DataFrame with historical data of all companies
for ticker in all_companies_and_index['cod_yfinance']:
    aux_df = pd.DataFrame()
    aux_df = yf.Ticker(ticker).history(period='max')
    aux_df['cod_yfinance'] = ticker

    historical_data = pd.concat([historical_data, aux_df], axis=0)

In [16]:
historical_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,cod_yfinance
0,1980-03-17,0.0000,0.426070,0.420489,0.420489,43200.0,0.0,0.0,IPG
1,1980-03-18,0.0000,0.416768,0.416768,0.416768,327600.0,0.0,0.0,IPG
2,1980-03-19,0.0000,0.416768,0.416768,0.416768,115200.0,0.0,0.0,IPG
3,1980-03-20,0.0000,0.420489,0.420489,0.420489,28800.0,0.0,0.0,IPG
4,1980-03-21,0.0000,0.420489,0.420489,0.420489,554400.0,0.0,0.0,IPG
...,...,...,...,...,...,...,...,...,...
4913755,2022-12-06,5.2800,5.280600,5.219792,5.280000,0.0,0.0,0.0,BRL=X
4913756,2022-12-07,5.2340,5.270356,5.210300,5.234000,0.0,0.0,0.0,BRL=X
4913757,2022-12-08,5.2040,5.248300,5.193944,5.204000,0.0,0.0,0.0,BRL=X
4913758,2022-12-09,5.2243,5.279800,5.214606,5.224300,0.0,0.0,0.0,BRL=X


In [ ]:
# Check if got all companies historical data
len(historical_data['cod_yfinance'].unique())

In [32]:
historical_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,cod_yfinance
0,1980-03-17,0.0000,0.426070,0.420489,0.420489,43200.0,0.0,0.0,IPG
1,1980-03-18,0.0000,0.416768,0.416768,0.416768,327600.0,0.0,0.0,IPG
2,1980-03-19,0.0000,0.416768,0.416768,0.416768,115200.0,0.0,0.0,IPG
3,1980-03-20,0.0000,0.420489,0.420489,0.420489,28800.0,0.0,0.0,IPG
4,1980-03-21,0.0000,0.420489,0.420489,0.420489,554400.0,0.0,0.0,IPG
...,...,...,...,...,...,...,...,...,...
4913755,2022-12-06,5.2800,5.280600,5.219792,5.280000,0.0,0.0,0.0,BRL=X
4913756,2022-12-07,5.2340,5.270356,5.210300,5.234000,0.0,0.0,0.0,BRL=X
4913757,2022-12-08,5.2040,5.248300,5.193944,5.204000,0.0,0.0,0.0,BRL=X
4913758,2022-12-09,5.2243,5.279800,5.214606,5.224300,0.0,0.0,0.0,BRL=X


In [33]:
historical_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4913760 entries, 0 to 4913759
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Date          object 
 1   Open          float64
 2   High          float64
 3   Low           float64
 4   Close         float64
 5   Volume        float64
 6   Dividends     float64
 7   Stock Splits  float64
 8   cod_yfinance  object 
dtypes: float64(7), object(2)
memory usage: 337.4+ MB


In [39]:
# Drop 2022-12-12
historical_data = historical_data.drop(historical_data[historical_data['Date'] == datetime.date(2022,12,12)].index, axis=0)

In [41]:
historical_data[historical_data['Date'] == datetime.date(2022,12,9)]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,cod_yfinance
10776,2022-12-09,32.000000,32.419998,31.840000,32.070000,1.766400e+06,0.0,0.0,IPG
21554,2022-12-09,75.519997,76.410004,75.099998,75.650002,1.160300e+06,0.0,0.0,OMC
27319,2022-12-09,13.900000,13.900000,13.490000,13.590000,3.683000e+05,0.0,0.0,TASA4.SA
33084,2022-12-09,13.310000,13.410000,13.090000,13.110000,4.267300e+06,0.0,0.0,EMBR3.SA
48427,2022-12-09,180.750000,183.889999,179.250000,179.539993,6.348800e+06,0.0,0.0,BA
...,...,...,...,...,...,...,...,...,...
4898778,2022-12-09,3954.169922,3977.020020,3933.040039,3934.379883,3.888260e+09,0.0,0.0,^GSPC
4904370,2022-12-09,1794.900024,1804.300049,1791.099976,1798.099976,7.130000e+02,0.0,0.0,GC=F
4907377,2022-12-09,17232.148438,17280.546875,17100.835938,17133.152344,2.032843e+10,0.0,0.0,BTC-USD
4909237,2022-12-09,1281.077271,1290.055420,1260.818726,1264.284790,5.706184e+09,0.0,0.0,ETH-USD


In [53]:
# Save DataFrame
historical_data.to_csv(f'{stock_path}historical_data_complete_rev5.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [26]:
# Load DataFrame
historical_data = pd.read_csv(f'{stock_path}historical_data_complete.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.')

In [ ]:
#Transform index to column Date
historical_data.reset_index(inplace=True)

In [29]:
# Transform date to datetime
historical_data['Date'] = pd.to_datetime(historical_data['Date'], utc=True) 

In [30]:
# Transform datetime to YYYY-MM-DD
historical_data['Date'] = historical_data['Date'].dt.date

In [18]:
len(historical_data['cod_yfinance'].unique())

735

### Rounding values

In [52]:
historical_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,cod_yfinance
0,1980-03-17,0.000,0.426,0.420,0.420,43200.0,0.0,0.0,IPG
1,1980-03-18,0.000,0.417,0.417,0.417,327600.0,0.0,0.0,IPG
2,1980-03-19,0.000,0.417,0.417,0.417,115200.0,0.0,0.0,IPG
3,1980-03-20,0.000,0.420,0.420,0.420,28800.0,0.0,0.0,IPG
4,1980-03-21,0.000,0.420,0.420,0.420,554400.0,0.0,0.0,IPG
...,...,...,...,...,...,...,...,...,...
4913754,2022-12-05,5.218,5.264,5.214,5.218,0.0,0.0,0.0,BRL=X
4913755,2022-12-06,5.280,5.281,5.220,5.280,0.0,0.0,0.0,BRL=X
4913756,2022-12-07,5.234,5.270,5.210,5.234,0.0,0.0,0.0,BRL=X
4913757,2022-12-08,5.204,5.248,5.194,5.204,0.0,0.0,0.0,BRL=X


In [51]:
historical_data['Open'] = historical_data['Open'].round(3)
historical_data['High'] = historical_data['High'].round(3)
historical_data['Low'] = historical_data['Low'].round(3)
historical_data['Close'] = historical_data['Close'].round(3)
historical_data['Dividends'] = historical_data['Dividends'].round(3)

## Dataset with indexes, gold, dolar (R$), bitcoin and ethereum historical data - Obsolete

In [ ]:
# Create empty DataFrame
indexes_historical_data = pd.DataFrame()

In [ ]:
# Creating DataFrame with historical data of all companies
for ticker in df_index['cod_yfinance']:
    aux_df = pd.DataFrame()
    aux_df = yf.Ticker(ticker).history(start='1927-12-30', end='2022-12-10')
    aux_df['cod_yfinance'] = ticker

    indexes_historical_data = pd.concat([indexes_historical_data, aux_df], axis=0)


In [ ]:
indexes_historical_data

In [ ]:
indexes_historical_data.to_csv(f'{stock_path}indexes_historical_data.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

## Testing yfinance historical data

In [ ]:
today = datetime.datetime.now()
today = today.strftime("%Y-%m-%d")

In [ ]:
teste = datetime.datetime.now()

In [ ]:
# Testinf yfinance historical data
test_df = pd.DataFrame()

for ticker in df_index['cod_yfinance']:
    aux_df = pd.DataFrame()
    aux_df = yf.Ticker(ticker).history(start='2022-12-08', end=today)
    aux_df['cod_yfinance'] = ticker

    test_df = pd.concat([test_df, aux_df], axis=0)

In [ ]:
test_df

In [ ]:
test_df.reset_index(inplace=True)

In [ ]:
test_df.info()

In [ ]:
# Transform date to datetime
test_df['Date'] = pd.to_datetime(test_df['Date'], utc=True) 

In [ ]:
# Transform datetime to YYYY-MM-DD
test_df['Date'] = test_df['Date'].dt.date

In [ ]:
test_df

# Working on companies sector and industry

In [60]:
all_companies = pd.read_csv(f'{stock_path}all_companies_and_indexes.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.')

In [61]:
all_companies['sector'].value_counts()

Industrials               104
Consumer Cyclical          98
Financial Services         93
Technology                 93
Healthcare                 81
Consumer Defensive         54
Utilities                  54
Real Estate                47
Basic Materials            37
Communication Services     33
Energy                     32
Index                       5
Crypto                      2
Gold                        1
Currency                    1
Name: sector, dtype: int64

In [62]:
all_companies['industry'].value_counts()

Utilities - Regulated Electric      30
Banks - Regional                    25
Specialty Industrial Machinery      21
Software - Application              19
Software - Infrastructure           18
                                    ..
REIT - Diversified                   1
Lumber & Wood Production             1
Other Industrial Metals & Mining     1
Security & Protection Services       1
Currency                             1
Name: industry, Length: 128, dtype: int64

In [ ]:
all_companies['industry'] = all_companies['industry'].str.replace("—"," - ")

In [64]:
all_companies[all_companies['industry'].str.contains('-')]

,ticker,name,sector,industry,cod_yfinance,country,type
59,BAC,Bank of America,Financial Services,Banks - Diversified,BAC,USA,Company
60,C,Citigroup,Financial Services,Banks - Diversified,C,USA,Company
61,JPM,JPMorgan Chase,Financial Services,Banks - Diversified,JPM,USA,Company
62,WFC,Wells Fargo,Financial Services,Banks - Diversified,WFC,USA,Company
63,ABCB4,ABC BRASIL,Financial Services,Banks - Regional,ABCB4.SA,Brazil,Company
...,...,...,...,...,...,...,...
717,ENGI11,ENERGISA,Utilities,Utilities - Renewable,ENGI11.SA,Brazil,Company
718,LIGT3,LIGHT S/A,Utilities,Utilities - Renewable,LIGT3.SA,Brazil,Company
719,MEGA3,OMEGAENERGIA,Utilities,Utilities - Renewable,MEGA3.SA,Brazil,Company
720,CEG,Constellation Energy,Utilities,Utilities - Renewable,CEG,USA,Company


In [ ]:
agf = ['QUAL3'
,'VVBR3'
,'AESB3'
,'WIZS3'
,'BRAP3'
,'BRSR6'
,'BRKM5'
,'BRAP4'
,'BRKM3'
,'BRSR3'
,'CGAS5'
,'TRPL4'
,'VIVT3'
,'TAEE11'
,'TAEE4'
,'TAEE3'
,'CGAS3'
,'ITSA4'
,'CSMG3'
,'ENAT3'
,'ITSA3'
,'SANB11'
,'SANB3'
,'SANB4'
,'GRND3'
,'BRSR5'
,'TRPL3'
,'SAPR4'
,'SAPR3'
,'BBSE3'
,'CMIG4'
,'ALUP11'
,'CLSC3'
,'CMIG4'
,'ELET3'
,'BBAS3'
,'BBDC4'
,'ITUB4'
,'AMBP3'
,'CSAN3'
,'CSAN4'
,'OPCT3'
,'SBSP3'
,'SAPR11'
,'VERZ34'
,'OIBR4'
,'TIMS3'
,'VIVT4'
,'TELB4'
,'TELB3'
,'ATTB34'
]


In [ ]:
all_companies[all_companies['ticker'].isin(agf)]

In [ ]:
all_companies.loc[all_companies['name'].isin(companies)]

In [ ]:
all_companies.loc[(all_companies['name'].isin(companies)) | all_companies['ticker'].isin(agf)].shape

In [83]:
# Perennial Industry
perennial=[
'Utilities - Diversified'
,'Utilities - Independent Power Producers'
,'Utilities - Regulated Electric'
,'Utilities - Regulated Gas'
,'Utilities - Regulated Water'
,'Utilities - Renewable'
,'Banks - Diversified'
,'Banks - Regional'
,'Insurance - Diversified'
,'Insurance - Life'
,'Insurance - Property & Casualty'
,'Insurance - Reinsurance'
,'Insurance - Specialty'
,'Insurance Brokers'
,'Healthcare Plans'
,'Telecom Services'
,'Waste Management'
,'Oil & Gas E&P'
,'Oil & Gas Equipment & Services'
,'Oil & Gas Integrated'
,'Oil & Gas Midstream'
,'Oil & Gas Refining & Marketing']

In [84]:
# Perennial subsector
dict_subsector = {
'Utilities - Diversified' : 'Utilities - Electricity'
,'Utilities - Independent Power Producers' : 'Utilities - Electricity'
,'Utilities - Regulated Electric' : 'Utilities - Electricity'
,'Utilities - Regulated Gas' :	'Utilities - Gas'
,'Utilities - Regulated Water' : 'Sanitation'
,'Utilities - Renewable' : 'Utilities - Electricity'
,'Banks - Diversified' : 'Banks'
,'Banks - Regional' : 'Banks'
,'Insurance - Diversified': 'Insurance'
,'Insurance - Life' : 'Insurance'
,'Insurance - Property & Casualty' : 'Insurance'
,'Insurance - Reinsurance' : 'Insurance'
,'Insurance - Specialty' : 'Insurance'
,'Insurance Brokers' : 'Insurance'
,'Healthcare Plans' : 'Health Insurance'
,'Telecom Services' : 'Telecom'
,'Waste Management' : 'Sanitation'
,'Oil & Gas E&P' : 'Oil & Gas'
,'Oil & Gas Equipment & Services' : 'Oil & Gas'
,'Oil & Gas Integrated' : 'Oil & Gas'
,'Oil & Gas Midstream' : 'Oil & Gas'
,'Oil & Gas Refining & Marketing' : 'Oil & Gas'
}

In [82]:
all_companies

,ticker,name,sector,industry,cod_yfinance,country,type,perennial_subsector,perennial
0,IPG,The Interpublic Group of Companies,Communication Services,Advertising Agencies,IPG,USA,Company,NaN,NaN
1,OMC,Omnicom Group,Communication Services,Advertising Agencies,OMC,USA,Company,NaN,NaN
2,TASA4,TAURUS ARMAS,Industrials,Aerospace & Defense,TASA4.SA,Brazil,Company,NaN,NaN
3,EMBR3,EMBRAER,Industrials,Aerospace & Defense,EMBR3.SA,Brazil,Company,NaN,NaN
4,BA,Boeing,Industrials,Aerospace & Defense,BA,USA,Company,NaN,NaN
...,...,...,...,...,...,...,...,...,...
730,SP500,S&P 500,Index,Index,^GSPC,USA,Index,NaN,NaN
731,GOLD,Gold (US$),Gold,Gold,GC=F,USA,Gold,NaN,NaN
732,BTC,Bitcoin (US$),Crypto,Crypto,BTC-USD,USA,Crypto,NaN,NaN
733,ETH,Ethereum (US$),Crypto,Crypto,ETH-USD,USA,Crypto,NaN,NaN


In [94]:
mask_perennial = all_companies['industry'].isin(perennial)

In [91]:
# Indetify if the company is in a perennial sector
all_companies.loc[mask_perennial, 'perennial'] = 'Yes'
all_companies.loc[~mask_perennial, 'perennial'] = 'No'

In [109]:
# Create a subsector for perennial companies
all_companies.loc[mask_perennial, 'perennial_subsector'] =  all_companies.loc[mask_perennial, 'industry'].map(dict_subsector)

c:\Users\Pedro\anaconda3\lib\site-packages\pandas\core\indexing.py:2115: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  new_ix = Index(new_ix)


In [110]:
# If the company is not perennial, subsector = industry
all_companies.loc[~mask_perennial, 'perennial_subsector'] = all_companies.loc[~mask_perennial, 'industry']

In [112]:
all_companies.isnull().sum()

ticker                 0
name                   0
sector                 0
industry               0
cod_yfinance           0
country                0
type                   0
perennial_subsector    0
perennial              0
dtype: int64

In [121]:
# Save modified DataFrame
all_companies_rev1.to_csv(f'{stock_path}all_companies_and_indexes_rev2.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [122]:
all_companies_rev2 = pd.read_csv(f'{stock_path}all_companies_and_indexes_rev1.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.')

In [123]:
all_companies_rev2

,ticker,name,sector,industry,cod_yfinance,country,type,perennial_subsector,perennial
0,IPG,The Interpublic Group of Companies,Communication Services,Advertising Agencies,IPG,USA,Company,Advertising Agencies,No
1,OMC,Omnicom Group,Communication Services,Advertising Agencies,OMC,USA,Company,Advertising Agencies,No
2,TASA4,TAURUS ARMAS,Industrials,Aerospace & Defense,TASA4.SA,Brazil,Company,Aerospace & Defense,No
3,EMBR3,EMBRAER,Industrials,Aerospace & Defense,EMBR3.SA,Brazil,Company,Aerospace & Defense,No
4,BA,Boeing,Industrials,Aerospace & Defense,BA,USA,Company,Aerospace & Defense,No
...,...,...,...,...,...,...,...,...,...
730,SP500,S&P 500,Index,Index,^GSPC,USA,Index,Index,No
731,GOLD,Gold (US$),Gold,Gold,GC=F,USA,Gold,Gold,No
732,BTC,Bitcoin (US$),Crypto,Crypto,BTC-USD,USA,Crypto,Crypto,No
733,ETH,Ethereum (US$),Crypto,Crypto,ETH-USD,USA,Crypto,Crypto,No


In [119]:
all_companies_rev1 = all_companies_rev1[['cod_yfinance', 'ticker', 'name', 'sector', 'industry', 'perennial_subsector', 'perennial', 'country', 'type']]

In [120]:
all_companies_rev1

,cod_yfinance,ticker,name,sector,industry,perennial_subsector,perennial,country,type
0,IPG,IPG,The Interpublic Group of Companies,Communication Services,Advertising Agencies,Advertising Agencies,No,USA,Company
1,OMC,OMC,Omnicom Group,Communication Services,Advertising Agencies,Advertising Agencies,No,USA,Company
2,TASA4.SA,TASA4,TAURUS ARMAS,Industrials,Aerospace & Defense,Aerospace & Defense,No,Brazil,Company
3,EMBR3.SA,EMBR3,EMBRAER,Industrials,Aerospace & Defense,Aerospace & Defense,No,Brazil,Company
4,BA,BA,Boeing,Industrials,Aerospace & Defense,Aerospace & Defense,No,USA,Company
...,...,...,...,...,...,...,...,...,...
730,^GSPC,SP500,S&P 500,Index,Index,Index,No,USA,Index
731,GC=F,GOLD,Gold (US$),Gold,Gold,Gold,No,USA,Gold
732,BTC-USD,BTC,Bitcoin (US$),Crypto,Crypto,Crypto,No,USA,Crypto
733,ETH-USD,ETH,Ethereum (US$),Crypto,Crypto,Crypto,No,USA,Crypto


## Get companies in AGF ranking using Selenium

In [ ]:
#Runs the webdriver
driver = webdriver.Chrome(options=chromeOptions)

In [ ]:
driver.implicitly_wait(wait)

In [ ]:
company_name = driver.find_elements(By.CSS_SELECTOR, '.text-secondary')

In [ ]:
companies = [i.text for i in company_name]

In [65]:
companies

NameError: name 'companies' is not defined

In [ ]:
# Get AGF companies ticker - not working
br_ticker = driver.find_elements(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "text-muted", " " ))]')

In [ ]:
all_companies.to_csv(f'{stock_path}all_companies_rev2.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

# Create SQL Database

In [ ]:
load_dotenv(r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\stocks_project/password.env'

In [ ]:
user = "root"
password = "password"
url_banco = "localhost"
nome_db = "bank"
conn_str = f"mysql+pymysql://{user}:{password}@{url_banco}/{nome_db}"

In [124]:
all_companies = pd.read_csv(f'{stock_path}all_companies_and_indexes_rev1.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.')

In [127]:
historical_data = pd.read_csv(f'{stock_path}historical_data_complete_rev5.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.')

In [130]:
historical_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,cod_yfinance
0,1980-03-17,0.000,0.426,0.420,0.420,43200.0,0.0,0.0,IPG
1,1980-03-18,0.000,0.417,0.417,0.417,327600.0,0.0,0.0,IPG
2,1980-03-19,0.000,0.417,0.417,0.417,115200.0,0.0,0.0,IPG
3,1980-03-20,0.000,0.420,0.420,0.420,28800.0,0.0,0.0,IPG
4,1980-03-21,0.000,0.420,0.420,0.420,554400.0,0.0,0.0,IPG
...,...,...,...,...,...,...,...,...,...
4913044,2022-12-05,5.218,5.264,5.214,5.218,0.0,0.0,0.0,BRL=X
4913045,2022-12-06,5.280,5.281,5.220,5.280,0.0,0.0,0.0,BRL=X
4913046,2022-12-07,5.234,5.270,5.210,5.234,0.0,0.0,0.0,BRL=X
4913047,2022-12-08,5.204,5.248,5.194,5.204,0.0,0.0,0.0,BRL=X


In [125]:
all_companies

,ticker,name,sector,industry,cod_yfinance,country,type,perennial_subsector,perennial
0,IPG,The Interpublic Group of Companies,Communication Services,Advertising Agencies,IPG,USA,Company,Advertising Agencies,No
1,OMC,Omnicom Group,Communication Services,Advertising Agencies,OMC,USA,Company,Advertising Agencies,No
2,TASA4,TAURUS ARMAS,Industrials,Aerospace & Defense,TASA4.SA,Brazil,Company,Aerospace & Defense,No
3,EMBR3,EMBRAER,Industrials,Aerospace & Defense,EMBR3.SA,Brazil,Company,Aerospace & Defense,No
4,BA,Boeing,Industrials,Aerospace & Defense,BA,USA,Company,Aerospace & Defense,No
...,...,...,...,...,...,...,...,...,...
730,SP500,S&P 500,Index,Index,^GSPC,USA,Index,Index,No
731,GOLD,Gold (US$),Gold,Gold,GC=F,USA,Gold,Gold,No
732,BTC,Bitcoin (US$),Crypto,Crypto,BTC-USD,USA,Crypto,Crypto,No
733,ETH,Ethereum (US$),Crypto,Crypto,ETH-USD,USA,Crypto,Crypto,No


In [ ]:
pd.concat